In [1]:
import pandas as pd
import numpy as np
import time
import random
import re
import string
import datetime
import nltk
import requests
from nltk import word_tokenize
from wordcloud import STOPWORDS

# Remove Punctuation

In [2]:
def remove_punc(s):
    import re
    import string
    pattern = re.compile('[\u4e00-\u9fa5]')
    s = re.sub(pattern,"",s)
    s = re.sub('[^A-Za-z.]',' ',s)
    punc = string.punctuation+"‘’“”—…‑"
    pattern = '[{}]'.format(re.escape(punc))
    s = re.sub(pattern, ' ', s)
    s = " ".join(s.split())
    s = s.lower()
    return s

# Remove @

In [3]:
def remove_at(s):
    s = str(s)
    s_l = s.split()
    ret_s = ''
    count = 0
    for i in range(len(s_l)):
        temp = s_l[i]
        if temp[0] == '@' and len(temp) != 1:
            count += 1
        else:
            ret_s += ' '+temp
    ret_s = ret_s[1:]
    return {'count': count,
            'return_string': ret_s}

# Turn Hashtag to Word

In [4]:
def get_hashtag(s):
    s = str(s)
    s_l = s.split()
    count = 0
    tag_l = []
    ret_s = ''
    for i in range(len(s_l)):
        temp = s_l[i]
        if temp[0] == '#' and len(temp) != 1:
            count += 1
            tag_l.append(temp[1:])
            ret_s += ' ' + temp[1:]
        else:
            ret_s += ' ' + temp
    ret_s = ret_s[1:]
    for i in range(len(tag_l)):
        tag_l[i] = remove_punc(tag_l[i])
    return {'tag_count': count,
            'tags': tag_l,
            'return_string': ret_s}

# Remove Hyperlink

In [5]:
def remove_hyperlink(s):
    s = str(s)
    ret_s = ''
    s_l = s.strip('\n').split()
    for i in range(len(s_l)):
        temp = s_l[i]
        if temp[:4] == 'http':
            continue
        else:
            ret_s += ' ' + temp
    ret_s = ret_s[1:]
    return ret_s

# Emoji

In [6]:
# !pip install emoji
import emoji

In [7]:
def emoji_process(s):
    s_demo = emoji.demojize(s)
    ret_s = remove_punc(s_demo)
    return ret_s

## Remove Stopping Words

In [8]:
def remove_stop_word(words):
    ret_l = []
    for i in range(len(words)):
        w = words[i]
        if w not in STOPWORDS:
            ret_l.append(w)
    return ret_l

# Pos & Neg

In [25]:
def get_words(url):
    word_list = requests.get(url).content.decode('latin-1').split('\n')
    idx = 0
    while idx < len(word_list):
        if ";" in word_list[idx] or word_list[idx] == "":
            word_list.pop(idx)
        else:
            idx += 1
    return word_list
# p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
# n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
# pos_word_list = get_words(p_url)
# neg_word_list = get_words(n_url)
# pos_word_df = pd.DataFrame(pos_word_list)
# neg_word_df = pd.DataFrame(neg_word_list)

In [10]:
pos_word_df = pd.read_csv('pos_word_list.csv',header=None, index_col=None)
neg_word_df = pd.read_csv('neg_word_list.csv',header=None, index_col=None)

In [26]:
def pos_neg_sentiment(s):
#     p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
#     n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
#     pos_word_list = get_words(p_url)
#     neg_word_list = get_words(n_url)
#     pos_word_df = pd.DataFrame(pos_word_list)
#     neg_word_df = pd.DataFrame(neg_word_list)
    pos_word_df = pd.read_csv('pos_word_list.csv',header=None, index_col=None)
    neg_word_df = pd.read_csv('neg_word_list.csv',header=None, index_col=None)
    
    pos_count = 0
    neg_count = 0
    words = word_tokenize(s)
    words = remove_stop_word(words)
    for i in range(len(words)):
        w = words[i]
        if w in pos_word_df:
            pos_count += 1
        if w in neg_word_df:
            neg_count += 1
    pos_pct = pos_count/len(words)
    neg_pct = neg_count/len(words)
    pos_neg_diff = pos_pct - neg_pct
    ret_d = {'pos rate': pos_pct,
            'neg rate': neg_pct,
            'pos neg diff': pos_neg_diff}
    return ret_d

# NRC

In [27]:
def nrc_emotion_dict():
    file_dir = "./data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    with open(file_dir, 'r') as f:
        lines = f.readlines()
    emotion_dict = dict()
    start_idx = 46
    for i in range(start_idx, len(lines)):
        line_l = lines[i].strip().split()
        word = line_l[0]
        emotion = line_l[1]
        t_or_f = line_l[2]
        if t_or_f == "1":
            if word in emotion_dict.keys():
                emotion_dict[word].append(emotion)
            else:
                emotion_dict[word] = [emotion]
    return emotion_dict

In [28]:
def nrc_sentiment(s):
    count_dict = {"anger": 0,
                "anticipation": 0,
                "disgust": 0,
                "fear": 0,
                "joy": 0,
                "negative": 0,
                "positive": 0,
                "sadness": 0,
                "surprise": 0,
                "trust": 0}
    emotion_dict = nrc_emotion_dict()
    words = word_tokenize(s)
    words = remove_stop_word(words)
    for i in range(len(words)):
        w = words[i]
        if w in emotion_dict.keys():
            e_l = emotion_dict[w]
            for j in range(len(e_l)):
                e = e_l[j]
                count_dict[e] += 1
    for k in count_dict.keys():
        count_dict[k] /= len(words)
    return count_dict

# Final - Combine Both

tweets

In [29]:
def get_dt_df(filename):
    twt_df = pd.read_excel(filename)
    twt_df = twt_df.drop(['Unnamed: 0', 'place', 'language', 'created_at'], axis = 1)
    twt_df['date'] = twt_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    twt_df['hour'] = twt_df['time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').hour)
    twt_df = twt_df.drop(['time'], axis = 1)
    twt_df = twt_df.sort_values(['date','hour']).reset_index().drop(['index'], axis = 1)

    dt_df = pd.DataFrame(data = None, 
                     index=twt_df.groupby(['date']).count().index, 
                     columns=['tweets', 'num', 'at_count', 'tag_count', 'reply_sum', 'retweet_sum', 'like_sum'])

    for i in range(len(twt_df)):
    # for i in range(2500):
        dt = twt_df.iloc[i]['date']
        hour = twt_df.iloc[i]['hour']
        tweet = twt_df.iloc[i]['tweet']
        reply = twt_df.iloc[i]['replies_count']
        retweet = twt_df.iloc[i]['retweets_count']
        like = twt_df.iloc[i]['likes_count']

        tweet_d = remove_at(tweet)
        tweet = tweet_d['return_string']
        at_count = tweet_d['count']

        tweet_d = get_hashtag(tweet)
        tweet = tweet_d['return_string']
        tag_count = tweet_d['tag_count']

        tweet = remove_hyperlink(tweet)

        if pd.isna(dt_df.loc[dt]['tweets']):
            dt_df.loc[dt]['tweets'] = str(tweet)
            dt_df.loc[dt]['num'] = 1
            dt_df.loc[dt]['reply_sum'] = reply
            dt_df.loc[dt]['retweet_sum'] = retweet
            dt_df.loc[dt]['like_sum'] = like
            dt_df.loc[dt]['at_count'] = at_count 
            dt_df.loc[dt]['tag_count'] = tag_count 

        else:
            dt_df.loc[dt]['tweets'] += ' ' + str(tweet)
            dt_df.loc[dt]['num'] += 1
            dt_df.loc[dt]['reply_sum'] += reply
            dt_df.loc[dt]['retweet_sum'] += retweet
            dt_df.loc[dt]['like_sum'] += like
            dt_df.loc[dt]['at_count'] += at_count 
            dt_df.loc[dt]['tag_count'] += tag_count 

    dt_df['tweets'] = dt_df['tweets'].apply(lambda x: emoji_process(x))
    return dt_df

sentiment

In [30]:
def get_sentiment_df(dt_df):
    col_names = ['date', 'tweets', 'pos', 'neg', 'pos - neg',
                                         'nrc anger', 'nrc anticipation', 'nrc disgust', 'nrc fear', 'nrc joy', 
                                         'nrc neg', 'nrc pos', 'nrc sadness', 'nrc surprise', 'nrc trust']
    col_names.extend(dt_df.columns.to_list()[1:])
    sentiment_df = pd.DataFrame(data = None,
                               index = range(len(dt_df)),
                               columns = col_names)
    for i in range(len(dt_df)):
        dt = dt_df.index[i]
        s = dt_df.iloc[i]['tweets']

        tweet = dt_df.iloc[i]['tweets']
        num = dt_df.iloc[i]['num']
        at_count = dt_df.iloc[i]['at_count']
        tag_count = dt_df.iloc[i]['tag_count']
        reply = dt_df.iloc[i]['reply_sum']
        retweet = dt_df.iloc[i]['retweet_sum']
        like = dt_df.iloc[i]['like_sum']

        sentiment_df.iloc[i]['date'] = dt
        sentiment_df.iloc[i]['tweets'] = tweet
        sentiment_df.iloc[i]['num'] = num
        sentiment_df.iloc[i]['at_count'] = at_count
        sentiment_df.iloc[i]['tag_count'] = tag_count
        sentiment_df.iloc[i]['reply_sum'] = reply
        sentiment_df.iloc[i]['retweet_sum'] = retweet
        sentiment_df.iloc[i]['like_sum'] = like


        pos_neg_d = pos_neg_sentiment(s)
        nrc_d = nrc_sentiment(s)
        sentiment_df.iloc[i][['pos','neg','pos - neg']] = list(pos_neg_d.values())
        sentiment_df.iloc[i][['nrc anger', 'nrc anticipation', 'nrc disgust', 'nrc fear', 'nrc joy', 
                    'nrc neg', 'nrc pos', 'nrc sadness', 'nrc surprise', 'nrc trust']] = list(nrc_d.values())
        
    return sentiment_df

In [37]:
def twitter_process(filename):
    
    dt_df = get_dt_df(filename)
    sentiment_df = get_sentiment_df(dt_df)
    
    NAME_MATCH = {'aa': 'AAL',
                 'dis': 'DIS',
                 'pep': 'PEP'}
    temp_filename = filename.split('\\')[-1].split('.')[0][:-4]
    y = filename.split('\\')[-1].split('.')[0][-4:]
    output_name = NAME_MATCH[temp_filename]
    output_dir = 'twitter\\' + output_name + f'_{y}_twt.csv'
    
    sentiment_df.to_csv(output_dir, 
                       header=True,
                       index=False,
                       encoding='utf-8-sig')
    return sentiment_df

# File Name

2018-2021 

In [32]:
# year_l = list(range(2018,2022))
# prefix = 'twitter\DIS_PEP_AAL_'
# ticker_l = ['aa', 'disney', 'pepsi']

In [33]:
# filename_l = []
# for i in range(len(year_l)):
#     y = year_l[i]
#     temp_prefix = prefix + str(y) + '\\'
#     for j in range(len(ticker_l)):
#         t = ticker_l[j]
#         temp_filename = temp_prefix + t + str(y) + '.xlsx'
#         filename_l.append(temp_filename)

In [38]:
year_l = [2017]
prefix = 'twitter\DIS_PEP_AAL_'
ticker_l = ['aa', 'dis', 'pep']

In [39]:
filename_l = []
for i in range(len(year_l)):
    y = year_l[i]
    temp_prefix = prefix + str(y) + '\\'
    for j in range(len(ticker_l)):
        t = ticker_l[j]
        temp_filename = temp_prefix + t + str(y) + '.xlsx'
        filename_l.append(temp_filename)

# Run for All Twitter Data

In [41]:
sentiment_l = []
print(f'---There are {len(filename_l)} files in total--- ')
for i in range(1,len(filename_l)):
    print(f'---Processing file No. {i}---')
    filename = filename_l[i]
    twitter_process(filename)

---There are 3 files in total--- 
---Processing file No. 1---
---Processing file No. 2---


In [ ]:
# filename = "twitter\DIS_PEP_AAL_2021\pepsi2021.xlsx"
# dt_df = get_dt_df(filename)
# sentiment_df = get_sentiment_df(dt_df)

In [ ]:
# filename = "twitter\DIS_PEP_AAL_2021\pepsi2021.xlsx"

In [ ]:
# sentiment_df

In [ ]:
# def get_sentiment(filename):
#     news = pd.read_csv(filename,
#                   header = 0,
#                   index_col = None)
#     sentiment_df = pd.DataFrame(data = None,
#                                index = range(len(news)),
#                                columns = ['date', 'news', 'pos', 'neg', 'pos - neg',
#                                          'nrc anger', 'nrc anticipation', 'nrc disgust', 'nrc fear', 'nrc joy', 
#                                          'nrc neg', 'nrc pos', 'nrc sadness', 'nrc surprise', 'nrc trust'])
#     for i in range(len(news)):
#         dt = news.iloc[i][0]
#         s = news.iloc[i][1]
#         if pd.isna(s):
#             sentiment_df.iloc[i]['date'] = dt
#         else:
#             pos_neg_d = pos_neg_sentiment(s)
#             nrc_d = nrc_sentiment(s)
#             sentiment_df.iloc[i]['date'] = dt
#             sentiment_df.iloc[i]['news'] = s
#             sentiment_df.iloc[i][['pos','neg','pos - neg']] = list(pos_neg_d.values())
#             sentiment_df.iloc[i][['nrc anger', 'nrc anticipation', 'nrc disgust', 'nrc fear', 'nrc joy', 
#                         'nrc neg', 'nrc pos', 'nrc sadness', 'nrc surprise', 'nrc trust']] = list(nrc_d.values())
    
#     sentiment_df.to_csv(filename[:3] + '_news_sentiment.csv', 
#                        header=True, 
#                        index=True)
#     return sentiment_df

In [ ]:
# filename = 'aal_dt_news.csv'
# get_sentiment(filename)

In [ ]:
# twt_df

In [ ]:
# news_dt_df.to_csv('aal_dt_news.csv',
#                  header=True,
#                  index=False,
#                  encoding='utf-8-sig')